<img src="images/GAlogo.png" style="float: left; margin: 15px; height: 100px">

# CAPSTONE PROJECT
## US TORNADOES AND HOW TO PREDICT THEIR MAGNITUDE
### Notebook for importing the mulitple csv files and copying them into local PostgreSQL database

In [83]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import requests
from bs4 import BeautifulSoup

%matplotlib inline

In [84]:
# To avoid display of warnings in Jupyter Notebook:
import warnings
warnings.filterwarnings('ignore')

In [63]:
# Link to all the csv files:
link = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
response = requests.get(link)
html = response.text

In [ ]:
# Making a soup out of the html and creating the list of file names:
soup = BeautifulSoup(html)
elements = soup.findAll("a",{"class":""})

In [71]:
StormEventDetails_allyears = []

# The first 6 elements and the last 2 have to be discarded:
for element in elements[6:-2]:
    
    # Selecting only the storm events tables (one for each year)
    if element.attrs['href'].startswith('StormEvents_details'):
        
        filename = element.attrs['href']
        StormEventDetails_url = link + filename
        
        # Creating an iterator in order to load the file in chunks of 1000 elements:
        iter_csv = pd.read_csv(StormEventDetails_url, compression='gzip', iterator=True,
                               chunksize=1000)
        
        # Concatenating the different chunks into a single dataframe, 
        # selecting the tornado events only:
        StormEventDetails_allyears.append(
            pd.concat([chunk[chunk['EVENT_TYPE'].map(lambda x: x.lower())
                             == 'tornado'] for chunk in iter_csvtttt]))

# Concatenating all the dataframes from the different years:
StormEventDetails = pd.concat(StormEventDetails_allyears)
        
print StormEventDetails.shape
StormEventDetails.head(2)

(67238, 51)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.2,35.17,-99.2,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.6,31.73,-98.6,NaN,NaN,PUB


In [73]:
# Doing the same for the other two tables (locations and fatalities):
StormEventFatalities_allyears = []
StormEventLocations_allyears = []

for element in elements[6:-2]:
    if element.attrs['href'].startswith('StormEvents_fatalities'):
        filename = element.attrs['href']
        StormEventFatalities_url = link+filename
        iter_csv = pd.read_csv(StormEventFatalities_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventFatalities_allyears.append(pd.concat(iter_csv))

    elif element.attrs['href'].startswith('StormEvents_locations'):
        filename = element.attrs['href']
        StormEventLocation_url = link+filename
        iter_csv = pd.read_csv(StormEventLocation_url, compression='gzip', iterator=True,
                               chunksize=1000)
        StormEventLocations_allyears.append(pd.concat(iter_csv))
        
StormEventFatalities = pd.concat(StormEventFatalities_allyears)
StormEventLocations = pd.concat(StormEventLocations_allyears)

print StormEventFatalities.shape, StormEventLocations.shape
StormEventFatalities.head(2)

(15991, 11) (1181011, 11)


,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,EVENT_ID,FATALITY_TYPE,FATALITY_DATE,FATALITY_AGE,FATALITY_SEX,FATALITY_LOCATION,EVENT_YEARMONTH
0,195001,13,525,1005198,9981922,D,01/13/1950 05:25:00,NaN,NaN,NaN,195001
1,195002,12,1200,1005199,10049525,D,02/12/1950 12:00:00,NaN,NaN,NaN,195002


In [82]:
StormEventLocations.head(2)

,YEARMONTH,EPISODE_ID,EVENT_ID,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,199603,2030059,5548852,1,NaN,NaN,LANGLEY,34.32,-93.83,3419.0,9350.0
1,199603,2030060,5548853,1,2.0,S,YELLVILLE,36.20,-92.68,3612.0,9241.0


In [78]:
# Creating engine connection to my local "storms" database, using sqlalchemy:
engine_local = create_engine('postgresql://localhost:5432/storms')

# Copying the first dataframe to local PostgreSQL:
StormEventDetails.to_sql('tornadoes_1950_mid2017', engine_local, index=False,
                         if_exists='replace')

In [79]:
# Copying the second dataframe
StormEventFatalities.to_sql('fatalities_1950_mid2017', engine_local, index=False,
                            if_exists='replace')

In [80]:
# Copying the third dataframe
StormEventLocations.to_sql('locations_1950_mid2017', engine_local, index=False,
                           if_exists='replace')

With the three tables now saved locally, we have a backup.<br>
Following is part 3 of the capstone, where the tables will be read directly from my local PostgreSQL.